Problem statement

The goal of this competition is to classify argumentative elements in student writing as "effective," "adequate," or "ineffective." You will create a model trained on data that is representative of the 6th-12th grade population in the United States in order to minimize bias. 

Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Load datasets

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith(('.csv')):
            print(os.path.join(dirname, filename))

Read files

In [ ]:
train = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/train.csv")
test = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/test.csv")
submission = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Analyse target

In [ ]:
sns.displot(train['discourse_effectiveness'])

Replace string with number

In [ ]:
replacers = {'Adequate':2,'Ineffective':3,'Effective':1}

train['discourse_effectiveness'] = train['discourse_effectiveness'].replace(replacers)
train

Boxplot

In [ ]:
plt.boxplot(train['discourse_effectiveness'])

In [ ]:
train['discourse_effectiveness'].value_counts()

In [ ]:
percentage=(train.discourse_effectiveness.value_counts() / len(train.discourse_effectiveness)) * 100
percentage

Define target

In [ ]:
target = train['discourse_effectiveness']

Combine train and test

In [ ]:
combi = train.drop(['discourse_effectiveness'], axis=1).append(test)
combi

Analyse discourse type

In [ ]:
sns.displot(combi['discourse_type'])

In [ ]:
combi['discourse_type'].value_counts()

In [ ]:
percentage_type=(combi.discourse_type.value_counts() / len(combi.discourse_type)) * 100
percentage_type

Define text

In [ ]:
text = combi['discourse_type'] + combi['discourse_text']

Count words to process

In [ ]:
count_words = text.str.findall(r'(\w+)').str.len()
print(count_words.sum())

Process text

In [ ]:
import re
from nltk.corpus import stopwords

""" Cleaning Tweets """
text = text.str.lower()

# removing special characters and numbers
text = text.apply(lambda x : re.sub("[^a-z\s]","",x) )

# remove hash tags
text = text.str.replace("#", " ")

#remove words less than 2 character
text = text.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# removing stopwords
stopwords = set(stopwords.words("english"))
text = text.apply(lambda x : " ".join(word for word in x.split() if word not in stopwords ))

count_words = text.str.findall(r'(\w+)').str.len()
print(count_words.sum())

Remove frequently used words

In [ ]:
most_freq_words = pd.Series(' '.join(text).lower().split()).value_counts()[:25]
text = text.apply(lambda x : " ".join(word for word in x.split() if word not in most_freq_words ))
print(most_freq_words)

count_words = text.str.findall(r'(\w+)').str.len()
print(count_words.sum())

Apostrophe dictionary

In [ ]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
apostrophe_dict

Function to replace words in dictionary

In [ ]:
def lookup_dict(txt, dictionary):
    for word in txt.split():
        if word.lower() in dictionary:
            if word.lower() in txt.split():
                txt = txt.replace(word, dictionary[word.lower()])
    return txt

In [ ]:
text = text.apply(lambda x: lookup_dict(x,apostrophe_dict))

Remove rare words

In [ ]:
type(text)

In [ ]:
from collections import Counter
from itertools import chain

# split words into lists
v = text.str.split().tolist()
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
text = [' '.join([j for j in i if c[j] > 1]) for i in v]

total_word = 0
for x,word in enumerate(text):
    num_word = len(word.split())
    total_word = total_word + num_word
print(total_word)

NLP with spacy

In [ ]:
!pip install -U spacy

Download english dictionary

In [ ]:
!python -m spacy download en

Natural language processing

In [ ]:
import spacy
import spacy.cli

spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')

In [ ]:
import spacy
import en_core_web_lg

nlp = en_core_web_lg.load()
document = nlp(text[0])
print("Document : ",document)
print("Tokens : ")
for token in document:
       print(token.text)

Token to vector

In [ ]:
document = nlp(text[0])
print(document)
for token in document:
    print(token.text, token.vector.shape)


Sentence to vector using pipe

In [ ]:
document = nlp.pipe(text)
text_vector = np.array([text.vector for text in document])
print(text_vector.shape)

Define X and y

In [ ]:
y = pd.get_dummies(target)

X = text_vector[: len(train)]
X_test = text_vector[len(train) :]

In [ ]:
y

Select model

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor

model =  MultiOutputRegressor(KNeighborsRegressor()).fit(X, y)
print(model.score(X, y))

Predict on X_test

In [ ]:
prediction = model.predict(X_test)
prediction

Prepare submission

In [ ]:
prediction = pd.DataFrame(prediction, columns=['Ineffective', 'Adequate', 'Effective'])
prediction['discourse_id'] = submission['discourse_id'] 
titles = ['discourse_id','Ineffective', 'Adequate', 'Effective']
prediction = prediction.reindex(columns=titles)
prediction

In [ ]:
submission = pd.DataFrame(prediction)
submission.to_csv('submission.csv', index=False)
submission